In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import requests
from bs4 import BeautifulSoup
import json
from typing import List
from Funcs.Utility import *
from tqdm import tqdm
from glob import glob
from time import sleep

from selenium import webdriver as webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import InvalidSessionIdException
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings("ignore")

uids = os.listdir(PATH_SENSOR)
columns = ["packageName","name","category","source"]
types = ["ACTIVITY_RESUMED","ACTIVITY_PAUSED"]



In [9]:
#Part 1: Get the list of uncategorized apps

if os.path.exists(os.path.join(PATH_SENSOR, "uncategorized_apps.csv")):
    os.remove(os.path.join(PATH_SENSOR, "uncategorized_apps.csv"))


# Make Entire app List from the data
app_list = []
for uid in uids:
    dfs = []
    for path in sorted(glob(os.path.join(PATH_SENSOR, uid, 'APP_USAGE_EVENT.csv'))):
        dfs.append(pd.read_csv(path, index_col = False, header = 0))
    if len(dfs) != 0: df = pd.concat(dfs)
    df = df.query('type in @types')
    app_list += df['packageName'].unique().tolist()
app_list = sorted(set(app_list))
print("# Total Apps:", len(app_list))

df = pd.DataFrame(app_list, columns=['packageName'])
df.drop(columns=[col for col in df.columns if 'Unnamed' in col], inplace=True)
print(df.head(), "\n", df.columns, "\n", df.shape, "\n")
df.to_csv(os.path.join(PATH_SENSOR, "uncategorized_apps.csv"))


# Total Apps: 836
                                   packageName
0  advanced.scientific.calculator.calc991.plus
1         air.kr.paikdabang.android.membership
2                                      android
3                          app.heymoon.release
4                                    app.intra 
 Index(['packageName'], dtype='object') 
 (836, 1) 



In [10]:
#Part 2: Categorize the apps, first stage:

print( "<"*20 + " First Extraction Began " + ">"*20 )
sleep(1)

if os.path.exists(os.path.join(PATH_SENSOR, "app_category_1.csv")):
    os.remove(os.path.join(PATH_SENSOR, "app_category_1.csv"))


def extractAppCategory1(apps: list)->pd.DataFrame:
    search_platform = "https://apkcombo.com"
    opts = FirefoxOptions()
    opts.add_argument("--headless")
    browser = webdriver.Firefox(options=opts)

    df = []
    app_list = pd.read_csv(os.path.join(PATH_SENSOR, "uncategorized_apps.csv"), index_col=0).to_numpy().tolist()
    for x in tqdm( app_list ):
        packageName = x[0].lower()
        browser.get(search_platform)

        row = [packageName, None, None, search_platform]
        # row = ["com.sktelecom.minit", None, None, search_platform]

        search = browser.find_element(By.NAME, "q")
        search.send_keys(row[0])
        browser.find_element(By.CLASS_NAME, "button-search").click()
        try:
            search_soup = BeautifulSoup(browser.page_source, 'html.parser')
            
            category_p_tag = search_soup.find('div', text=lambda text: text and 'Category' in text)

            category = category_p_tag.find_next_sibling("div").text
            row[2] = category.upper()

            title_tag = search_soup.find('div', class_='app_name').findChild('h1').findChild('a').text
            row[1] = title_tag.upper()
        except:
            print("Error")
        print(row)
        df.append(row)
    browser.close()
    return pd.DataFrame(df, columns=columns)

df = extractAppCategory1(pd.read_csv(os.path.join(PATH_SENSOR, 'uncategorized_apps.csv'), index_col=0).to_numpy().tolist())
df.drop(columns=[col for col in df.columns if 'Unnamed' in col], inplace=True)
df.to_csv(os.path.join(PATH_SENSOR, "app_category_1.csv"))
print(df.isna().sum())

<<<<<<<<<<<<<<<<<<<< First Extraction Began >>>>>>>>>>>>>>>>>>>>


  0%|          | 1/836 [00:02<32:05,  2.31s/it]

['advanced.scientific.calculator.calc991.plus', 'SCIENTIFIC CALCULATOR PLUS 991', 'EDUCATION', 'https://apkcombo.com']


  0%|          | 2/836 [00:03<26:26,  1.90s/it]

['air.kr.paikdabang.android.membership', '빽다방', 'FOOD & DRINK', 'https://apkcombo.com']


...

100%|█████████▉| 835/836 [36:09<00:06,  6.69s/it]

['world.mnetplus', 'MNET PLUS', 'ENTERTAINMENT', 'https://apkcombo.com']


100%|██████████| 836/836 [36:14<00:00,  2.60s/it]

['wsj.reader_sp', 'THE WALL STREET JOURNAL.', 'NEWS & MAGAZINES', 'https://apkcombo.com']


packageName     0
name           67
category       67
source          0
dtype: int64


In [18]:
#Part 3: Categorize the apps, second stage:

print( "<"*20 + " Second Extraction Began " + ">"*20 )
sleep(1)

if os.path.exists(os.path.join(PATH_SENSOR, "app_category_2.csv")):
    os.remove(os.path.join(PATH_SENSOR, "app_category_2.csv"))

def extractAppCategory2(apps: list)->pd.DataFrame:
    dics = []
    opts = FirefoxOptions()
    opts.add_argument("--headless")
    browser = webdriver.Firefox(options=opts)
    for x in tqdm(apps):
        id = x[0]
        row = [id, None, None, "https://play.google.com"]
        search_platform = f"https://www.google.com/search?q={id}&start=1"

        browser.get(search_platform)

        anchor_tags = browser.find_elements(By.TAG_NAME, "a")

        # Extract href attributes from anchor tags
        links = [tag.get_attribute("href") for tag in anchor_tags]

        # Filter out None values (some anchor tags may not have an href attribute)
        links = [link for link in links if link is not None and "https://play.google.com" in link]

        # Print the list of links
        target_link = None
        for link in links:
            if id.lower() in link.lower():
                target_link = link

        if target_link:
            browser.get(target_link)
            sleep(4)
            try:
                soup = BeautifulSoup(browser.page_source, 'html.parser')

                # Find the h1 tag with itemprop="name"
                h1_with_itemprop_name = soup.find('h1', {'itemprop': 'name'})

                category = soup.find("div", {'class': 'Uc6QCc'}).find("a").attrs['href'].split("/")[-1]
                row[1] = h1_with_itemprop_name.text
                row[2] = category
            except:
                print("Error")
            print(row)
        dics.append(row)
        # Close the WebDriver
        sleep(4)
    browser.quit()
    return pd.DataFrame(dics, columns=columns)

df = pd.read_csv(os.path.join(PATH_SENSOR, 'app_category_1.csv'), index_col=0)
res = extractAppCategory2(df[df.isna().any(axis=1)].to_numpy().tolist())
df.set_index('packageName', inplace=True)
res.set_index('packageName', inplace=True)
df.update(res)
df.reset_index(inplace=True)
print(df.head(), "\n", df.columns, "\n", df.shape, "\n")
df.to_csv(os.path.join(PATH_SENSOR, "app_category_1.csv"))
print(df.isna().sum())

<<<<<<<<<<<<<<<<<<<< Second Extraction Began >>>>>>>>>>>>>>>>>>>>


  0%|          | 0/67 [00:00<?, ?it/s]

['android', '내 안드로이드', 'TOOLS', 'https://play.google.com']


  1%|▏         | 1/67 [00:10<11:47, 10.72s/it]

Error
['com.agentsoft.s1', None, None, 'https://play.google.com']


...

['com.sktelecom.danawa.main', '다나와: 최저가 가격비교, 조립PC, 자동차, 여행', 'SHOPPING', 'https://play.google.com']


 63%|██████▎   | 42/67 [05:43<02:48,  6.74s/it]

['com.susoft.cjonmart', 'CJ더마켓', 'FOOD_AND_DRINK', 'https://play.google.com']


100%|██████████| 67/67 [07:48<00:00,  6.99s/it]


                                   packageName  \
0  advanced.scientific.calculator.calc991.plus   
1         air.kr.paikdabang.android.membership   
2                                      android   
3                          app.heymoon.release   
4                                    app.intra   

                             name          category                   source  
0  SCIENTIFIC CALCULATOR PLUS 991         EDUCATION     https://apkcombo.com  
1                             빽다방      FOOD & DRINK     https://apkcombo.com  
2                         내 안드로이드             TOOLS  https://play.google.com  
3                   헤이문 - 월경 건강 앱  HEALTH & FITNESS     https://apkcombo.com  
4                           INTRA             TOOLS     https://apkcombo.com   
 Index(['packageName', 'name', 'category', 'source'], dtype='object') 
 (836, 4) 

packageName     0
name           40
category       40
source          0
dtype: int64


In [13]:
def extractAppCategory2(app: pd.DataFrame) -> pd.DataFrame:
    for idx, row in tqdm(app.iterrows(), total=app.shape[0]):
        package_id = app.iloc[idx, app.columns.get_loc('packageName')]
        if not row.isna().any(): continue
        try:
            search_url = f'https://apkpure.net/search?q={package_id}'
            search_response = requests.get(search_url)
            search_soup = BeautifulSoup(search_response.text, 'html.parser')
            url = search_soup.find('a', class_='first-info').get('href')
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            category_p_tag = soup.find('p', text=lambda text: text and 'Category' in text)
            if category_p_tag:
                category = ' '.join(category_p_tag.find_next_sibling("p").text.split()[1:-1])
                app['category'][idx] = category.upper()
            title_tag = soup.find('div', class_="title_link" ).find_next("h1")
            if title_tag:
                app['name'][idx] = title_tag.text
            app['source'][idx] = 'apkpure.net'
            # print(title_tag.text, category)
        except Exception as e:
            pass
            # print(e)
        # input()
    return app
df = pd.read_csv(os.path.join(PATH_SENSOR, "app_category_1.csv"), index_col = False, header = 0)
print(df.isna().sum())
print("-"*50)
df = extractAppCategory2(df)
print(df.isna().sum())
df.drop(columns=[col for col in df.columns if 'Unnamed' in col], inplace=True)
df.to_csv(os.path.join(PATH_SENSOR, "app_category_1.csv"))

Unnamed: 0      0
packageName     0
name           40
category       40
source          0
dtype: int64
--------------------------------------------------


100%|██████████| 836/836 [01:15<00:00, 11.14it/s]


Unnamed: 0     0
packageName    0
name           7
category       7
source         0
dtype: int64


In [19]:
apps = pd.read_csv(os.path.join(PATH_SENSOR, 'app_category.csv'), index_col=0)

print("[AppUsageEvent.csv]")

for uid in tqdm(uids):
    if ".csv" in uid: continue
    df = pd.read_csv(os.path.join(PATH_SENSOR, uid, 'APP_USAGE_EVENT.csv'), index_col=0)
    package_category_mapping = dict(zip(apps['packageName'], apps['category']))
    df['category'] = df['packageName'].map(package_category_mapping)
    df.to_csv(os.path.join(PATH_SENSOR, uid, 'APP_USAGE_EVENT.csv'))

[AppUsageEvent.csv]


100%|██████████| 26/26 [00:17<00:00,  1.52it/s]
